# Importando as bibliotecas


In [1]:
import sys
sys.path.append('/home/hurias/Documentos/Disciplina_NLP/Atv_1')


import json
import glob
import os
import io
from base import get_stats, merge
from BPE import BPE_Tokenizer
import numpy as np
import random
import math


# Preparação dos dados

Etapa é feito a tokenização o corpus e dividir os dados em treino e teste (80% e 20%)


In [2]:
pasta_json = r"/home/hurias/Downloads/corpus" 
count = 0

merge_texto = io.StringIO()

# Carregar os arquivos JSON e extrair os textos
for filename in os.listdir(pasta_json):
    if filename.endswith('.json'):
        file_path = os.path.join(pasta_json, filename)
        try:
            with open(file_path, 'r') as file:
                data = json.load(file)
                count += 1
                if 'text' in data:
                    merge_texto.write(data['text'] + " ")
                else:
                    print(f"A chave 'text' não encontrada no arquivo: {filename}")
        except json.JSONDecodeError:
            print(f"Erro ao decodificar o JSON no arquivo: {filename}")

print(f"Arquivos lidos: {count}")


Arquivos lidos: 10000


In [3]:
# Obtenha o texto combinado do corpus
corpus = merge_texto.getvalue()
print(f"Tamanho do corpus: {len(corpus)} caracteres")

Tamanho do corpus: 71033261 caracteres


In [4]:
# Verifique se o corpus não está vazio
if len(corpus.strip()) == 0:
    print("Erro: O corpus está vazio!")
    sys.exit(1)  # Encerra a execução se o corpus estiver vazio

In [5]:
# Instanciar o BPE Tokenizer
num_merges = 100
tokenizer = BPE_Tokenizer(num_merges)

In [6]:
# Verifique o vocabulário após o treinamento
tokenizer.train(corpus)
print("Vocabulário após treinamento:", tokenizer.vocab)

Tamanho do texto em caracteres: 71033261
Tamanho do texto em tokens: 73037124
Tamanho da de tokens após BPE: 44430539
Taxa de compressão: 1.64X
Vocabulário após treinamento: {0: b'\x00', 1: b'\x01', 2: b'\x02', 3: b'\x03', 4: b'\x04', 5: b'\x05', 6: b'\x06', 7: b'\x07', 8: b'\x08', 9: b'\t', 10: b'\n', 11: b'\x0b', 12: b'\x0c', 13: b'\r', 14: b'\x0e', 15: b'\x0f', 16: b'\x10', 17: b'\x11', 18: b'\x12', 19: b'\x13', 20: b'\x14', 21: b'\x15', 22: b'\x16', 23: b'\x17', 24: b'\x18', 25: b'\x19', 26: b'\x1a', 27: b'\x1b', 28: b'\x1c', 29: b'\x1d', 30: b'\x1e', 31: b'\x1f', 32: b' ', 33: b'!', 34: b'"', 35: b'#', 36: b'$', 37: b'%', 38: b'&', 39: b"'", 40: b'(', 41: b')', 42: b'*', 43: b'+', 44: b',', 45: b'-', 46: b'.', 47: b'/', 48: b'0', 49: b'1', 50: b'2', 51: b'3', 52: b'4', 53: b'5', 54: b'6', 55: b'7', 56: b'8', 57: b'9', 58: b':', 59: b';', 60: b'<', 61: b'=', 62: b'>', 63: b'?', 64: b'@', 65: b'A', 66: b'B', 67: b'C', 68: b'D', 69: b'E', 70: b'F', 71: b'G', 72: b'H', 73: b'I', 74: b

In [7]:
# Verifique se o vocabulário foi gerado corretamente
if len(tokenizer.vocab) == 0:
    print("Erro: O vocabulário não foi gerado corretamente.")
    sys.exit(1)  # Encerra a execução se o vocabulário não foi gerado

In [8]:
# Use o tokenizador para segmentar o corpus
tokens = tokenizer.encode(text=corpus)

# Treinamento do corpus e Bigram

In [9]:
# Divisão em treino (80%) e teste (20%)
split_index = int(len(tokens) * 0.8)
random.shuffle(tokens)
train_tokens = tokens[:split_index]
test_tokens = tokens[split_index:]

print("Total de tokens no treino:", len(train_tokens))
print("Total de tokens no teste:", len(test_tokens))


Total de tokens no treino: 35544431
Total de tokens no teste: 8886108


In [10]:
from bigram import BigramModel

In [11]:
# Treine o modelo bigrama

bigram_model = BigramModel()
bigram_model.train(train_tokens)

# Perplexity do modelo 

In [12]:
# Função para calcular a perplexidade
def calculate_perplexity(model, tokens):
    perplexity = 0
    N = len(tokens)
    for i in range(len(tokens) - 1):
        prob = model.bigram_prob(tokens[i], tokens[i + 1])
        if prob > 0:
            perplexity += -math.log(prob)
    perplexity = math.exp(perplexity / N)
    return perplexity

# Calcule a perplexidade no conjunto de teste
perplexity = calculate_perplexity(bigram_model, test_tokens)
print("Perplexidade do modelo:", perplexity)


Perplexidade do modelo: 124.84411359166064


# Geração de Texto 



In [16]:
# Função para gerar texto a partir de um token inicial
def generate_text(model, last_token, length=20):
    text = [last_token]
    
    for _ in range(length - 1):
        # Obtenha as próximas palavras e suas probabilidades condicionais
        next_words = list(model.bigram_counts[last_token].keys())
        next_probs = [model.bigram_prob(last_token, word) for word in next_words]
        
        # Verificar se os próximos tokens estão no vocabulário
        next_words = [word for word in next_words if word in tokenizer.vocab]
        if not next_words: 
            break 
        
        # Normaliza as probabilidades
        next_probs = np.array(next_probs)
        next_probs /= next_probs.sum()

        # Seleciona a próxima palavra com base na probabilidade condicional
        last_token = np.random.choice(next_words, p=next_probs)
        text.append(last_token)
    
    # Decodificar os tokens de volta para palavras (strings)
    decoded_text = [tokenizer.decode([token]) for token in text]
    
    # Juntar as palavras para formar uma string de texto gerado
    return " ".join(decoded_text)



In [19]:
# Exemplo de como gerar texto a partir do fragmento
text_frag = "Flamengo é o melhor time "
print("Fragmento de texto:", text_frag)

# Tente codificar as palavras individualmente
frag_tokens = []
for word in text_frag.split():
    try:
        tokens = tokenizer.encode(word)
        print(f"Tokens para '{word}': {tokens}")
        frag_tokens.extend(tokens)
    except Exception as e:
        print(f"Erro ao codificar a palavra '{word}': {e}")

# Se houver tokens, pegue o último
if frag_tokens:
    last_token = frag_tokens[-1]
    print("Último token:", last_token)
else:
    print("Nenhum token foi gerado.")

# Gere texto a partir do último token
generated_text = generate_text(bigram_model, last_token)
print("Texto gerado:", generated_text)  # Agora isso deve gerar o texto legível


Fragmento de texto: Flamengo é o melhor time 
Tokens para 'Flamengo': [70, 108, 296, 262, 103, 111]
Erro ao codificar a palavra 'é': min() iterable argument is empty
Erro ao codificar a palavra 'o': min() iterable argument is empty
Tokens para 'melhor': [109, 289, 104, 264]
Tokens para 'time': [116, 332, 101]
Último token: 101
Texto gerado: e que  st   do  v st b e  í de  c est d s  o  u t ur r
